In [2]:
# !pip install --upgrade pip
!yes | pip uninstall scipy
!pip install scipy==1.11.4
!pip install paddlepaddle paddleocr
!yes| pip uninstall transformers==4.36.0
!pip install -q transformers==4.31.0
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 trl==0.4.7

Found existing installation: SciPy 1.11.4
Uninstalling SciPy-1.11.4:
  Would remove:
    /opt/conda/lib/python3.10/site-packages/scipy-1.11.4.dist-info/*
    /opt/conda/lib/python3.10/site-packages/scipy/*
  Would not remove (might be manually added):
    /opt/conda/lib/python3.10/site-packages/scipy/HACKING.rst.txt
    /opt/conda/lib/python3.10/site-packages/scipy/INSTALL.rst.txt
    /opt/conda/lib/python3.10/site-packages/scipy/LICENSE.txt
    /opt/conda/lib/python3.10/site-packages/scipy/LICENSES_bundled.txt
    /opt/conda/lib/python3.10/site-packages/scipy/_build_utils/__init__.py
    /opt/conda/lib/python3.10/site-packages/scipy/_build_utils/_fortran.py
    /opt/conda/lib/python3.10/site-packages/scipy/_build_utils/compiler_helper.py
    /opt/conda/lib/python3.10/site-packages/scipy/_build_utils/setup.py
    /opt/conda/lib/python3.10/site-packages/scipy/_build_utils/system_info.py
    /opt/conda/lib/python3.10/site-packages/scipy/_build_utils/tempita.py
    /opt/conda/lib/python3.

In [3]:
!pip install watchdog
!pip install --no-dependencies --quiet streamlit
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -o ngrok-stable-linux-amd64.zip
!pip install --quiet pyngrok
!pip install --no-dependencies --quiet protobuf==3.20.*   #==4.21.12
!pip install --no-dependencies --quiet validators

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 2.0 MB/s eta 0:00:0000:01
--2023-12-21 11:05:03--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.202.168.65, 54.161.241.46, 54.237.133.81, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.202.168.65|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13921656 (13M) [application/octet-stream]
Saving to: 'ngrok-stable-linux-amd64.zip'

ngrok-stable-linux- 100%[===================>]  13.28M  6.07MB/s    in 2.2s    

2023-12-21 11:05:06 (6.07 MB/s) - 'ngrok-stable-linux-amd64.zip' saved [13921656/13921656]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [4]:
!ngrok authtoken "2ZnyVSH3qJtktP8Tg6sdoLIWNse_2qfvJ2TVM34kc54y28XgK" 

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml                                


In [10]:
%%writefile myapp.py

import streamlit as st

import requests
from PIL import Image
import torch
import numpy as np
import pandas as pd
import os
from paddleocr import PaddleOCR
from PIL import Image
import logging
from transformers import AutoProcessor, Blip2ForConditionalGeneration
from io import BytesIO

from peft import LoraConfig, PeftModel
from trl import SFTTrainer

from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer

logger = logging.getLogger('PaddleOCR')
logging.basicConfig(level=logging.WARNING)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


@st.cache_resource
def load_paddle_ocr():
    return PaddleOCR(use_angle_cls=True, lang='en')

@st.cache_resource
def load_blip2_model():
    processor = AutoProcessor.from_pretrained("Salesforce/blip2-opt-2.7b")
    model = Blip2ForConditionalGeneration.from_pretrained("Salesforce/blip2-opt-2.7b", torch_dtype=torch.float16)
    return processor, model.to(device)

@st.cache_resource
def load_peft_model(peft_model_dir):
    trained_model = AutoPeftModelForCausalLM.from_pretrained(
        peft_model_dir,
        low_cpu_mem_usage=True,
        torch_dtype=torch.float16,
        load_in_4bit=True,
    )
    tokenizer = AutoTokenizer.from_pretrained(peft_model_dir)
    return trained_model, tokenizer

# Usage in your main app
ocr = load_paddle_ocr()
blip2_processor, blip2_model = load_blip2_model()
peft_model, peft_tokenizer = load_peft_model("/kaggle/input/aaaaaa/llama-2-7b-custom-20231215T130254Z-001/llama-2-7b-custom")

def fetch_image(image_url):

    response = requests.get(image_url)
    image = None
    if(response.status_code == 200):
        image = Image.open(BytesIO(response.content)).convert("RGB")
    else:
        black_image_size = (224, 224)  
        image = Image.new("RGB", black_image_size, "black")
    return image

ocr = PaddleOCR(use_angle_cls=True, lang='en')

def perform_ocr_with_paddleocr(image):

    try:
        img_array = np.array(image)
        st.write(img_array)
        result = ocr.ocr(img_array, cls=True)
        st.write(result)
    except Exception as e:
        print(f"Error during OCR: {e}")
        return []

    if result is None:
        print("No OCR results.")
        return []

    extracted_texts = []
    for entry in result:
        if entry:  # Check if entry is not None
            for item in entry:
                text = item[1][0]  
                extracted_texts.append(text)
    return extracted_texts

def create_prompt(caption, texts, company, username, date):
    
    prompt = f"As a social media expert, your task is to create an engaging and human-like description for a Twitter post. Analyze the tone, style, and content of the post to develop a description that effectively resonates with its intended audience. Pay special attention to any text content present in the images associated with the post, as this text should be seamlessly integrated into your description to enhance relevance and engagement. Utilize the provided data from the Twitter post as a reference for your description. DON'T PREDICT MORE THAN FIVE EMOJIS PLEASE "

    data_description = (f"Caption for Image of post: {caption}, " 
                        f"Text content present on the Image: {texts},"
                       f"Inferred company: {company}, " 
                       f"Username: {username}, " 
                       f"Date and time: {date}, "
                   )

    prompt += data_description
    return prompt

import os
import torch
from datasets import load_dataset

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)

logging.set_verbosity(logging.CRITICAL)
pipe = pipeline(task="text-generation", model=peft_model, tokenizer=peft_tokenizer, max_length=700)

def main():
    
    st.title("Task 2 - Content Simulation :)")


    with st.form("user_input_form"):

        img_url = st.text_input("Enter the Image URL:")
        if img_url:
            img = fetch_image(img_url)
        else:
            img = None
            
        if img is not None:
            img = img.resize((224, 224))
            st.image(img, caption="Processed Image", use_column_width=True)
        else:
            black_image_size = (224, 224)  
            img = Image.new("RGB", black_image_size, "black")

        company = st.text_input("Enter the Inferred Company : ")
        username = st.text_input("Enter the User Handle : ")
        date = st.text_input("Enter the Date and Time (e.g., 2018-01-29 10:51:17): ")
        
        image = img
        inputs = blip2_processor(image, return_tensors="pt").to(device, torch.float16)
        generated_ids = blip2_model.generate(**inputs, max_new_tokens=700)
        caption = blip2_processor.batch_decode(generated_ids, skip_special_tokens=True)[0].strip()
        
        text = ""
        text = perform_ocr_with_paddleocr(image)

        prompt = create_prompt(caption, text, company, username, date)
        
        st.write(f"**Prompt**")
        st.write(prompt)

        submitted = st.form_submit_button("Generate Tweet")
        if submitted:
            result = pipe(prompt, num_return_sequences = 1)
            st.success(f"{result[0]['generated_text'].split('[/INST]')[1].strip()}")
            
if __name__ == "__main__":
    main()

Overwriting myapp.py


In [11]:
from pyngrok import ngrok
import threading

def run_ngrok():
    ngrok_tunnel = ngrok.connect(8501)
    print(f'Public URL: {ngrok_tunnel.public_url}')
    ngrok_tunnel.block_until_close()

# Start ngrok in a separate thread
ngrok_thread = threading.Thread(target=run_ngrok)
ngrok_thread.start()

Exception in thread Thread-8 (run_ngrok):
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_42/567140438.py", line 7, in run_ngrok
AttributeError: 'NgrokTunnel' object has no attribute 'block_until_close'


Public URL: https://95f1-35-201-165-232.ngrok-free.app


In [ ]:
!streamlit run --server.port 8501 myapp.py > /dev/null

100%|█████████████████████████████████████| 4.00M/4.00M [00:01<00:00, 2.12MiB/s]
100%|█████████████████████████████████████| 10.2M/10.2M [00:00<00:00, 16.4MiB/s]
100%|█████████████████████████████████████| 2.19M/2.19M [00:00<00:00, 5.52MiB/s]
tokenizer_config.json: 100%|███████████████████| 904/904 [00:00<00:00, 5.58MB/s]
vocab.json: 100%|████████████████████████████| 798k/798k [00:00<00:00, 1.03MB/s]
merges.txt: 100%|█████████████████████████████| 456k/456k [00:00<00:00, 783kB/s]
tokenizer.json: 100%|██████████████████████| 2.11M/2.11M [00:00<00:00, 2.16MB/s]
special_tokens_map.json: 100%|█████████████████| 548/548 [00:00<00:00, 2.84MB/s]
config.json: 100%|█████████████████████████| 6.96k/6.96k [00:00<00:00, 27.4MB/s]
pytorch_model.bin.index.json: 100%|███████████| 122k/122k [00:00<00:00, 315kB/s]
pytorch_model-00001-of-00002.bin:   0%|             | 0.00/10.0G [00:00<?, ?B/s]
pytorch_model-00001-of-00002.bin:   0%|    | 10.5M/10.0G [00:00<02:31, 65.9MB/s]
pytorch_model-00001-of-00002